# GAIA-Coleta360 - Segmentação de Telhados
Este notebook demonstra como treinar e testar um modelo de segmentação de telhados em imagens de satélite, utilizando PyTorch e FPN com ResNet34.

In [ ]:
# 📦 Instalação de dependências (caso necessário)
# !pip install torch torchvision segmentation-models-pytorch albumentations opencv-python matplotlib tqdm

## 🔧 Utils (Dataset, Augmentações, Dice Score)

In [ ]:

import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

class RoofDataset(Dataset):
    def __init__(self, images_dir, masks_dir, augmentation=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.augmentation = augmentation
        self.image_names = sorted(os.listdir(images_dir))

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_names[idx])
        mask_path = os.path.join(self.masks_dir, self.image_names[idx])

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if self.augmentation:
            augmented = self.augmentation(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

def get_training_augmentation(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.GaussNoise(p=0.2),
        A.RandomCrop(height=img_size, width=img_size, p=1.0),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2()
    ])

def get_validation_augmentation(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2()
    ])

def dice_score(preds, targets, smooth=1e-6):
    preds = preds.view(-1)
    targets = targets.view(-1)
    intersection = (preds * targets).sum()
    return (2. * intersection + smooth) / (preds.sum() + targets.sum() + smooth)


## 🚀 Treinamento do Modelo

In [ ]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
from utils import RoofDataset, get_training_augmentation, get_validation_augmentation, dice_score
from tqdm import tqdm

# Diretórios
DATA_DIR = 'data/train'
MASK_DIR = 'data/train_masks'
MODEL_DIR = 'models'

# Hiperparâmetros
BATCH_SIZE = 16
NUM_EPOCHS = 50
LEARNING_RATE = 1e-4
IMG_SIZE = 256

# Dataset e DataLoader
train_dataset = RoofDataset(
    images_dir=DATA_DIR,
    masks_dir=MASK_DIR,
    augmentation=get_training_augmentation(IMG_SIZE)
)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Modelo
model = smp.FPN(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Otimizador e função de perda
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.BCEWithLogitsLoss()

# Treinamento
model.train()
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    for images, masks in loop:
        images = images.to(device)
        masks = masks.to(device).unsqueeze(1)

        preds = model(images)
        loss = loss_fn(preds, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} Loss: {epoch_loss/len(train_loader):.4f}")

# Salvando o modelo
os.makedirs(MODEL_DIR, exist_ok=True)
torch.save(model.state_dict(), os.path.join(MODEL_DIR, 'fpn_resnet34.pth'))
print("\nModelo salvo com sucesso!")


## 🔍 Inferência e Avaliação

In [ ]:

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
from utils import RoofDataset, get_validation_augmentation, dice_score
from torchvision.utils import save_image

# Diretórios
IMAGE_DIR = 'data/test'
MASK_DIR = 'data/test_masks'
MODEL_PATH = 'models/fpn_resnet34.pth'
RESULTS_DIR = 'results'
os.makedirs(RESULTS_DIR, exist_ok=True)

# Parâmetros
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 256
BATCH_SIZE = 8

# Dataset e DataLoader
test_dataset = RoofDataset(
    images_dir=IMAGE_DIR,
    masks_dir=MASK_DIR,
    augmentation=get_validation_augmentation(IMG_SIZE)
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Modelo
model = smp.FPN(
    encoder_name="resnet34",
    encoder_weights=None,
    in_channels=3,
    classes=1
)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# Avaliação e salvamento
all_dice_scores = []
with torch.no_grad():
    for i, (images, masks) in enumerate(test_loader):
        images = images.to(device)
        masks = masks.to(device).unsqueeze(1)

        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5

        # Dice score
        dice = dice_score(preds.float(), masks.float())
        all_dice_scores.append(dice.item())

        # Salvando resultados
        for j in range(images.size(0)):
            result_path = os.path.join(RESULTS_DIR, f'result_{i*BATCH_SIZE + j}.png')
            save_image(preds[j].float(), result_path)

# Resultado final
mean_dice = np.mean(all_dice_scores)
print(f"\nDice Score médio no conjunto de teste: {mean_dice:.4f}")
